In [486]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [487]:
import numpy as np

from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [488]:
def get_n_values(p_pt,i):
    '''
    This function calculates the n values for
    every p_pt and gas i
    
    Inputs:
    p_pt: a 1D array of the p_pt values (in kPa) on which we are calculating n
    i: integer index to characterize a gas
    
    Returns:
    n_values: a 1D array of the n values corresponding to each p_pt
    '''
    a = a_data[i]
    b = b_data[i]
    n_values=[]
    for p in p_pt:
        poly = np.poly1d([a*b/V**3, -a/V, R*T_pt + p*b, -p*V])
        solutions = np.roots(poly) #This statement uses numpy roots to solve third order polynomial
        for sol in solutions: #This control seq. select the only real solution.
            if abs(sol.imag) < 1e-5:
                n_values.append(sol.real)
    return n_values

In [489]:
def get_p_values(n_values,T,i):
    '''
    This functions calculates the p values for
    every n in n_values, temperature T and gas i by
    solving the Van der Waals equation of state for n.
    
    Inputs:
    n_values: a 1d array of the n values of each measurement
    T: float temperature (in K) for which we are calculating pressure
    i: integer index to characterize a gas
    
    Returns:
    p: a 1D array of the p values corresponding to gas i for each value of n
    '''
    a = a_data[i]
    b = b_data[i]
    p = []
    for n in n_values:
        p.append(n*R*T/(V-n*b)-a*n**2/V**2)
    return p

In [490]:
def get_empirical_temperature(p_pt,T,n_of_gases):
    '''
    This functions calculates the empirical temperature of thermal state T
    for which p_pt and p have been previously calculated, by applying the 
    definiton of empirical temperature.
    
    Inputs:
    p_pt: a 1D array of the p_pt values for which we are calculating Theta
    T: float temperature (in K) to measure
    n_of_gases: number of gases to consider in the plot.
    
    Returns:
    Theta: a 2D array of the values of empirical temperature for each gas (first index)
    and for each value of p_pt (second index).
    '''
    
    Theta = []
    for i in range(n_of_gases):
        n_values = get_n_values(p_pt,i)
        p = get_p_values(n_values,T,i)
        Theta.append(p/p_pt * 273.16) 
    return Theta

In [491]:
def update_figure(change):
    '''
    This function updates the plot figure whenever the T slider
    value is changed or checkbox are turned on/off
    
    WARNING: Funtzio honek checkbox-a klikatzen duen bakoitzan Theta
    berkalkulatzen du. Hori ez da oso efizientea. Oraingoz ez dut beste
    modurik.
    '''
    obj = change.owner
    
    op=[0.0,0.0,0.0,0.0,0.0,0.0]
    
    if Hidrogen_check_121_003.value == True:
        op[0] = 1.0
    if Helium_check_121_003.value == True:
        op[1] = 1.0
    if Nitrogen_check_121_003.value == True:
        op[2] = 1.0
    if Oxygen_check_121_003.value == True:
        op[3] = 1.0
    if Neon_check_121_003.value == True:
        op[4] = 1.0
    if Argon_check_121_003.value == True:
        op[5] = 1.0
    
    T = T_slider_121_002.value
    y_values = get_empirical_temperature(p_pt,T,c)
    scale_y = bqs.LinearScale(min = T-0.2, max = T+0.2)
    
    axis_y.scale=scale_y
    data.y = y_values
    data.scales = {'x': scale_x, 'y': scale_y}
    data.opacities = op
    

In [492]:
# Fixed Parameters
V = 1 #Volume in L (Doesn't affect the final results)
T_pt = 273.16 # Temperature of fixed point (triple point of water) in K. 
R = 8.31446 # Gas constant in L*kPa/K/mol

# Gas data
labels = ["Hidrogenoa","Helioa","Nitrogenoa", "Oxigenoa", "Neona", "Argona"]
latex = ["$H_2$", "$He$", "$N_2$", "$O_2$", "$Ne$", "$Ar$"]
a_data = [24.7100, 3.4600, 137.00, 138.2, 21.3500, 135.500, 820.000]
b_data = [0.02661, 0.0238, 0.0387, 0.03186, 0.01709, 0.03201, 0.01696]
colors = ['#0079c4','#f09205','#21c400','#FF0000', '#00FF00', '#0000FF', '#000000']
c=6 #Cutoff: number of gases to consider
opacities = [1.0, 1.0, 1.0, 0.0, 0.0, 0.0]

#Initial state:
p_pt = np.linspace(1,120,30)
T = 373.16
Theta = get_empirical_temperature(p_pt, T, c)





#######################################
#######CREATE THE FIGURES##############
#######################################


fig_121_001 = bq.Figure(title='Tenperatura enpirikoak',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-left',
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
    )

scale_x = bqs.LinearScale(min = 0.0, max = max(p_pt))
scale_y = bqs.LinearScale(min = T-0.2, max = T+0.2)

axis_x = bqa.Axis(scale=scale_x,
                tick_format='.2f',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values = np.linspace(0, max(p_pt), 7),
                #num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='p_pt (kPa)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y = bqa.Axis(
                scale=scale_y,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='Theta (K)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_121_001.axes = [axis_x,axis_y]





#######################################
#########CREATE THE MARKS##############
#######################################


x_values = [ p_pt for i in range(c)]
y_values = Theta

data = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x, 'y': scale_y}, 
                opacities = opacities,
                visible = True, #True, #t == '1.00',
                colors = colors,
                labels = labels,
                display_legend = True
)

fig_121_001.marks = [data]





#######################################
#########CONTROL WIDGETS###############
#######################################

T_slider_121_002 = widgets.FloatSlider(
    min=200.0,
    max=400.0,
    step=1.0,
    value=T,
    description='T (ideal gas scale)',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='90%')
)

T_slider_121_002.observe(update_figure, 'value')

Hidrogen_check_121_003 = widgets.Checkbox(
    value=True,
    description='Hidrogenoa',
    disabled=False
)


Helium_check_121_003 = widgets.Checkbox(
    value=True,
    description='Helioa',
    disabled=False
)

Nitrogen_check_121_003 = widgets.Checkbox(
    value=True,
    description='Nitrogenoa',
    disabled=False
)

Oxygen_check_121_003 = widgets.Checkbox(
    value=False,
    description='Oxigenoa',
    disabled=False
)

Neon_check_121_003 = widgets.Checkbox(
    value=False,
    description='Neona',
    disabled=False
)

Argon_check_121_003 = widgets.Checkbox(
    value=False,
    description='Argona',
    disabled=False
)



Hidrogen_check_121_003.observe(update_figure, 'value')
Helium_check_121_003.observe(update_figure, 'value')
Nitrogen_check_121_003.observe(update_figure, 'value')
Oxygen_check_121_003.observe(update_figure, 'value')
Neon_check_121_003.observe(update_figure, 'value')
Argon_check_121_003.observe(update_figure, 'value')



#######################################
############  LAYOUT  #################
#######################################



right_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='80%', align_items='center'))

right_block_121_000.children = [widgets.VBox([fig_121_001], layout=widgets.Layout(width='60%', align_self='center')),
                              widgets.VBox([T_slider_121_002], layout=widgets.Layout(width='50%' , align_self='center'))]

left_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_items='center'))
left_block_121_000.children = [widgets.VBox([Hidrogen_check_121_003, Helium_check_121_003, Nitrogen_check_121_003,
                                            Oxygen_check_121_003, Neon_check_121_003, Argon_check_121_003
                                            ],
                                            layout=widgets.Layout(width='100%', align_self='center'))]


main_block_121_000 = widgets.HBox([], layout=widgets.Layout(width='100%', align_items='center'))
main_block_121_000.children = [left_block_121_000, right_block_121_000]

main_block_121_000

In [493]:
#Hobetzeko gauzak
# 1. update_figure funtzioa optimizatu Theta ez kalkulatzeko checkbox bat zapaltzen den bakoitzean
# 2. Gasen koloreak ondo aukeratu (O2, Ne eta Ar batez ere).
# 3. Gas batzuen kurba ia identikoa da. Beste batzuk aukeratu?
# 4. Layouta txukundu. Left block-a oso urruti dago, nola gerturatu?
# 5. LaTeX inplementatzeko modurik?